In [85]:
from tensorflow.python.keras._impl.keras.preprocessing import sequence
from tensorflow.python.keras._impl.keras.layers import Input, Embedding, Dense, Conv2D, MaxPool2D, BatchNormalization
from tensorflow.python.keras._impl.keras.layers import Reshape, Flatten, Dropout, Concatenate, dot, add

from tensorflow.python.keras._impl.keras.optimizers import Adam
from tensorflow.python.keras._impl.keras.models import Model
from tensorflow.python.keras._impl.keras.layers import LSTM
from tensorflow.python import keras
from tensorflow.python.keras._impl.keras.layers.wrappers import TimeDistributed, Bidirectional
import tensorflow as tf

from tensorflow.python.keras._impl.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras._impl.keras.preprocessing.sequence import pad_sequences
from tensorflow.python.keras._impl.keras.utils.data_utils import get_file

import numpy as np
import json
from sklearn.model_selection import train_test_split

# Initial global var

In [86]:
Q1_TRAINING_DATA_FILE = 'q1_train.npy'
Q2_TRAINING_DATA_FILE = 'q2_train.npy'
LABEL_TRAINING_DATA_FILE = 'label_train.npy'
WORD_EMBEDDING_MATRIX_FILE = 'word_embedding_matrix.npy'
NB_WORDS_DATA_FILE = 'nb_words.json'
MODEL_WEIGHTS_FILE = 'question_pairs_weights.h5'
SENT_EMB_DIM = 300
MAX_SEQ_LEN = 25
EMBEDDING_DIM = 300
VALIDATION_SPLIT = 0.1
TEST_SPLIT = 0.1
RNG_SEED = 13371447
NB_EPOCHS = 25
DROPOUT = 0.1
BATCH_SIZE = 32
DATA_PATH = './data/'

# Load Dataset

In [87]:
q1_data = np.load(open(DATA_PATH + Q1_TRAINING_DATA_FILE, 'rb'))
q2_data = np.load(open(DATA_PATH + Q2_TRAINING_DATA_FILE, 'rb'))
labels = np.load(open(DATA_PATH + LABEL_TRAINING_DATA_FILE, 'rb'))
word_embedding_matrix = np.load(open(DATA_PATH+WORD_EMBEDDING_MATRIX_FILE, 'rb'))
with open(DATA_PATH+NB_WORDS_DATA_FILE, 'r') as f:
    nb_words = json.load(f)['nb_words']

# Split train and test dataset

In [81]:
X = np.stack((q1_data, q2_data), axis=1)
y = labels
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SPLIT, random_state=RNG_SEED)
Q1_train = X_train[:,0]
Q2_train = X_train[:,1]
Q1_test = X_test[:,0]
Q2_test = X_test[:,1]

In [88]:
class Attention(tf.keras.Model):
    
    def __init__(self):
        super(Attention, self).__init__()
        
        self.MAX_SEQ_LEN = MAX_SEQ_LEN
        
        self.q1_input = Input(shape=(MAX_SEQ_LEN, ))
        self.q2_input = Input(shape=(MAX_SEQ_LEN, ))
        
        self.embedding = Embedding(nb_words + 1,
                 EMBEDDING_DIM, 
                 weights=[word_embedding_matrix],
                 input_length=MAX_SEQ_LEN, 
                 trainable=False)        
        
        self.Bidirectional = Bidirectional(LSTM(SENT_EMB_DIM, return_sequences=True), merge_mode='sum')
                
    def predict(self):
        
        q1_input = self.q1_input      
        q1_encoded = self.embedding(q1_input)
        q1_bi = self.Bidirectional(q1_encoded)
        
        q2_input = self.q2_input
        q2_encoded = self.embedding(q2_input)
        q2_bi = self.Bidirectional(q2_encoded)
                
        attention = dot([q1_encoded, q2_encoded], [1,1])
        attention = Flatten()(attention)
        attention = Dense((MAX_SEQ_LEN * SENT_EMB_DIM))(attention)
        attention = Reshape((MAX_SEQUENCE_LENGTH, SENT_EMB_DIM))(attention)
        
        print(attention)
        
        merged = add([q1_encoded,attention])
        merged = Flatten()(merged)
        merged = Dense(200, activation='relu')(merged)
        merged = Dropout(DROPOUT)(merged)
        merged = BatchNormalization()(merged)
        
        is_duplicate = Dense(1, activation='sigmoid')(merged)
        
        model = Model(inputs=[self.q1_input,self.q2_input], outputs=is_duplicate)
        model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
        print(model.summary())
        
        return model

In [89]:
att = Attention()

In [90]:
model = att.predict()

Tensor("reshape_4/Reshape:0", shape=(?, 25, 300), dtype=float32)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_30 (InputLayer)           (None, 25)           0                                            
__________________________________________________________________________________________________
input_31 (InputLayer)           (None, 25)           0                                            
__________________________________________________________________________________________________
embedding_14 (Embedding)        (None, 25, 300)      28678800    input_30[0][0]                   
                                                                 input_31[0][0]                   
__________________________________________________________________________________________________
dot_5 (Dot)                     (None, 300, 

In [63]:
Q1_train.shape

(363858, 25)

In [61]:
q1_input = Input(shape=(MAX_SEQ_LEN, ))
q1_input(Q1_train)

TypeError: 'Tensor' object is not callable

In [ ]:

printprint(("Starting training at""Starti , datetime.datetime.now())
t0 = time.time()
callbacks = [ModelCheckpoint(MODEL_WEIGHTS_FILE, monitor='val_acc', save_best_only=True)]
history = model.fit([Q1_train, Q2_train],
                    y_train,
                    epochs=NB_EPOCHS,
                    validation_split=VALIDATION_SPLIT,
                    verbose=2,
                    batch_size=BATCH_SIZE,
                    callbacks=callbacks)
t1 = time.time()
print("Training ended at", datetime.datetime.now())
print("Minutes elapsed: %f" % ((t1 - t0) / 60.))

In [ ]:
https://github.com/bradleypallen/keras-quora-question-pairs/blob/master/quora-question-pairs-data-prep.ipynb
https://github.com/Smerity/keras_snli
https://github.com/bradleypallen/keras-quora-question-pairs/blob/master/quora-question-pairs-training-attention.ipynb
https://github.com/bradleypallen/keras-quora-question-pairs/blob/master/quora-question-pairs-training.ipynb
http://www.wildml.com/2016/01/attention-and-memory-in-deep-learning-and-nlp/
